In [4]:
import pandas as pd
import numpy as np

In [5]:
path_in = "/Users/timholdsworth/code/scaling-science/Data/100_most_impactful_papers.csv"
def get_data(path_in):
    df = pd.read_csv(path_in)
    return df

In [6]:
# Returns a series of len(df) where each value is the column the data starts in for each row
def get_start_columns(df):
    
    time_since_pub = df.loc[:, 'total_years_pub']
    # Because there are 3 extra column at the end
    start_col_pos = time_since_pub + 3
    # Because we want the last certain number of columns
    start_cols = -start_col_pos
    
    return start_cols

In [74]:
# Iterates through rows of a dataframe and updates scores by multiplying them by exp decay factor
def exp_decay(df, start_col):
    
    for index, row in df.iterrows():
        print(index)
        
        # Get the nondecayed scores 
        start = start_col[index]
        impact_scores = df.iloc[index, start:-3].reset_index(drop=True)
        
        # Generate a series of decay coefficients
        time = np.arange(len(impact_scores))
        decay_list = [np.exp(-t / 25) for t in time]
        decay_series = pd.Series(decay_list)
        
        # Multiply the decay coefficeints by the nondecayed scores
        decayed_score = decay_series.multiply(impact_scores)
        
        # Decayed frame is for one paper and correctly updates with each iteration
        # Turn the decayed_score into a df with column names matching and back to year-indexed series
        decayed_frame = decayed_score.to_frame()
        cols = list(df.columns.values)
        year = int(cols[start])
        year_list = ['' + str(year + t) + '' for t in time]
        year_series = pd.Series(year_list)
        decayed_frame['years'] = year_series
        decayed_frame = decayed_frame.set_index('years').T 
        #decayed_series1 = decayed_frame.squeeze()
        
        # This seems to mis
        df.update(decayed_frame)
        
        #print(impact_scores)
        #print(decayed_frame)
        #print(decayed_series1.index)
        #print(df.columns.values)
        
        #df.iloc[index, start:-3].update(decayed_series1)
        
        # Debugging notes: impact score, index, decayed_frame updates fine with iterations
        # The dataframe only updates after the first iteration
        
        
        # TODO figure out why this works only to update the first row - do I need to reset something 
        # at the top of the method
        # df.update(decayed_frame) works to update values in the first row but misallocates the values
        #df.update(decayed_frame)
        #print(df.iloc[index, -7:-3])
        
        #print('Printing decayed frame')
        #print(decayed_frame)
        #print(type(decayed_series_indexed))
        
    
    return df

In [75]:
# Take in a csv with a bunch of data that has year scores and apply exponential decay
def main():
    
    df = get_data(path_in).head(5)
    start_cols = get_start_columns(df)
    df_decayed = exp_decay(df, start_cols)
    return df_decayed

df_decayed = main()
df_decayed

0
1
2
3
4


,title,1900,1901,1902,1903,1904,1905,1906,1907,1908,...,1944,1945,1946,1947,1948,1949,1950,score_sum,total_years_pub,time_weighted_score
0,Zur Frage des Einflusses der Milz auf den Eise...,0.15,0.15,0.144118,0.138467,0.133038,0.127822,0.12281,0.117994,0.113368,...,0.218683,0.210108,0.201869,0.193954,0.186349,0.179042,0.172022,7.680,23,0.334
1,Zur Pathogenese des Sogenannten „Röntgenkaters“,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.278,0.278,0.278,0.278,0.278,0.278,0.278,7.770,28,0.277
2,Substituirte Stickstoffbromide und ihre Bezieh...,0.15,0.278,0.278,0.278,0.278,0.278,0.278,0.278,0.278,...,0.278,0.278,0.278,0.278,0.278,0.278,0.278,14.025,51,0.275
3,Über die Acetylierung von löslicher Stärke,NaN,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,...,0.386,0.386,0.386,0.386,0.386,0.386,0.386,13.735,50,0.275
4,The action of young's glycotropic factor of th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.278,0.278,0.278,0.278,0.278,0.278,0.278,3.480,13,0.268


In [ ]:
path_out = '/Users/timholdsworth/code/scaling-science/Data/100_most_impactful_papers_decayed.csv'
def write_to_csv(df):
    df.to_csv(path_out, index=False)

In [ ]:
write_to_csv(df_decayed)

In [ ]:
df = get_data(path_in).head(10)
cols = list(df.columns.values)
year = cols[-10]
ind = int(year)
ind

In [99]:
# Play with the df.update until it works
df0 = pd.DataFrame({'A': [1, 2, 3], 'B': [400, 500, 600]})
df1 = pd.DataFrame({'A': [4, 5, 6]})
df0.update(df1)

,A,B
0,4,400
1,5,500
2,6,600


In [102]:
for index, row in df0.iterrows():
    print(index)
    df00 = pd.DataFrame({'A': [index+8, index+9, index+10]})
    df0.update(df00)
    #df00 = df1.squeeze()
    #df0.iloc[index, :].update(df00)
#new_df = pd.DataFrame({'B': [4, 5, 6],'C': [7, 8, 9]})
#df.update(new_df)
df0

0
1
2


,A,B
0,10,400
1,11,500
2,12,600


In [ ]:
# Method to calculate decay_scores 
def calc_decay_scores(df, start_col):
    
    # For a given paper, calculate a series with the decay scores 
    
    for index, row in df.iterrows():
        print(index)
        
        # Get the nondecayed scores 
        start = start_col[index]
        impact_scores = df.iloc[index, start:-3].reset_index(drop=True)
        
        # Generate a series of decay coefficients
        time = np.arange(len(impact_scores))
        decay_list = [np.exp(-t / 25) for t in time]
        decay_series = pd.Series(decay_list)
        
        # Multiply the decay coefficeints by the nondecayed scores
        decay_score = decay_series.multiply(impact_scores)
        return decay_score
    

In [ ]:
# Method to update the dataframe with the impact scores
def update_df_with_decay_scores(df, decay_scores, start_col):
    
    # For all papers:
    # Take in a series of decay scores for a given paper
    # Add correct years as index and turn to a df
    # Transpose main df
    # Update main df with scores
    
    
    # Where each column represents a paper
    for columns in df:
        # Calculate the decay scores for each row
        decay_scores = calc_decay_scores(df, start_col)
        
        # Decayed frame is for one paper and correctly updates with each iteration
        # Turn the decayed_score into a df with column names matching and back to year-indexed series
        decay_frame = decay_score.to_frame()
        cols = list(df.columns.values)
        year = int(cols[start])
        year_list = ['' + str(year + t) + '' for t in time]
        year_series = pd.Series(year_list)
        decay_frame['years'] = year_series
        decay_frame = decay_frame.set_index('years') 
        #decayed_series1 = decayed_frame.squeeze()
        
        # Update the values
        df.update(decay_frame)
        
    return df
    
    

        